# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 12hSleeping whole brain AKR/J strain (A) Expression values and Group 2 be 12hSD whole brain AKR/J strain (A) Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_ZT12_AK_'   # define a prefix to add to column names (making indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE9442.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 87)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 87)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239868_AK_S_ZT6.CEL',
       'GSM239869_AK_S_ZT6.CEL', 'GSM239870_AK_S_ZT6.CEL', 'avg-AK_S_ZT6',
       'log_fold-AK_S_ZT6_vs_AK_SD_ZT6', 'fold-AK_S_ZT6_vs_AK_SD_ZT6',
       'rawp-AK_S_ZT6_vs_AK_SD_ZT6', 'adjp-AK_S_ZT6_vs_AK_SD_ZT6',
       'GSM239871_AK_S_ZT12.CEL', 'GSM239872_AK_S_ZT12.CEL',
       'GSM239873_AK_S_ZT12.CEL', 'avg-AK_S_ZT12',
       'log_fold-AK_S_ZT12_vs_AK_SD_ZT12', 'fold-AK_S_ZT12_vs_AK_SD_ZT12',
       'rawp-AK_S_ZT12_vs_AK_SD_ZT12', 'adjp-AK_S_ZT12_vs_AK_SD_ZT12',
       'GSM239880_AK_SD_ZT6.CEL', 'GSM239881_AK_SD_ZT6.CEL',
       'GSM239882_AK_SD_ZT6.CEL', 'avg-AK_SD_ZT6', 'GSM239883_AK_SD_ZT12.CEL',
       'GSM239884_AK_SD_ZT12.CEL', 'GSM239885_AK_SD_ZT12.CEL',
       'avg-AK_SD_ZT12', 'GSM239891_B6_S_ZT6.CEL',
  

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='AK_S_ZT12.CEL'
sd_filt ='AK_SD_ZT12.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM239871_AK_S_ZT12.CEL,GSM239872_AK_S_ZT12.CEL,GSM239873_AK_S_ZT12.CEL
Probesets,,,
1427138_at,6.23562,6.17613,6.52825
1425600_a_at,7.31109,7.54674,7.15165
1457168_at,4.94453,4.48763,4.46599
1450135_at,5.09171,5.31121,5.19808
1424014_at,8.14794,8.12018,8.17982


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM239883_AK_SD_ZT12.CEL,GSM239884_AK_SD_ZT12.CEL,GSM239885_AK_SD_ZT12.CEL
Probesets,,,
1427138_at,6.18968,6.41150,6.44727
1425600_a_at,7.51928,7.24088,7.60428
1457168_at,4.66249,4.53819,4.64995
1450135_at,5.13482,4.70765,5.07921
1424014_at,8.16308,8.25014,8.18309


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at      0.036150
1425600_a_at    0.118320
1457168_at     -0.015840
1450135_at     -0.226440
1424014_at      0.049457
Name: 430AV2_ZT12_AK_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at      0.217994
1425600_a_at    0.608412
1457168_at     -0.080353
1450135_at     -1.246642
1424014_at      1.283419
Name: 430AV2_ZT12_AK_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at      0.185295
1425600_a_at    0.517150
1457168_at     -0.068300
1450135_at     -1.059646
1424014_at      1.090906
Name: 430AV2_ZT12_AK_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.670627
1425600_a_at    0.697514
1457168_at      0.667205
1450135_at      0.796176
1424014_at      0.803930
Name: 430AV2_ZT12_AK_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_ZT12_AK_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_ZT12_AK_Enrich,430AV2_ZT12_AK_poolStDev,430AV2_ZT12_AK_Cohens_d,430AV2_ZT12_AK_J,430AV2_ZT12_AK_Hedges_g,430AV2_ZT12_AK_Var_d,430AV2_ZT12_AK_Var_g,430AV2_ZT12_AK_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1429017_at,Smcr8,"Smith-Magenis syndrome chromosome region, cand...",ENSMUSG00000049323,237782,NaN,NaN,NaN,NaN,NaN,"mmu-miR-1192(miRanda), mmu-miR-124(TargetScan|...",...,3.402851e-02,0.336713,0.222593,0.007854,28.339892,0.85,24.088909,67.595792,48.837960,6.988416
1446085_at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.677669e-02,0.316520,0.179970,0.008441,21.321644,0.85,18.123397,38.551042,27.853128,5.277606
1420998_at,Etv5,ets variant gene 5,ENSMUSG00000013089|ENSMUSG00000044690,104156,NaN,organ morphogenesis // positive regulation of ...,transcription factor activity // DNA binding /...,nucleus,PluriNetWork:WP1763(WikiPathways),"mmu-miR-101(TargetScan), mmu-miR-101a(miRanda)...",...,4.542745e-06,0.735293,0.525913,0.025143,20.916729,0.85,17.779219,37.125794,26.823386,5.179130
1452352_at,Ctla2b,cytotoxic T lymphocyte-associated protein 2 beta,ENSMUSG00000074874,13025,NaN,NaN,protein binding // cysteine-type peptidase act...,NaN,NaN,"mmu-miR-101a(miRanda), mmu-miR-101b(miRanda), ...",...,1.521859e-02,0.645520,0.423050,0.023113,18.303855,0.85,15.558277,28.585925,20.653331,4.544594
1448629_at,Hps4,Hermansky-Pudlak syndrome 4 homolog (human),ENSMUSG00000042328,192232,NaN,protein targeting // organelle organization //...,protein dimerization activity // protein homod...,cytoplasm // membrane fraction // melanosome /...,NaN,"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,2.931206e-05,0.454997,0.331347,0.020302,16.320544,0.85,13.872462,22.863346,16.518767,4.064329
1423796_at,Sfpq,splicing factor proline/glutamine rich (polypy...,ENSMUSG00000028820,71514,NaN,transcription // RNA splicing // mRNA processi...,RNA binding // nucleic acid binding // nucleot...,nucleus // nuclear matrix,mRNA processing:WP310(WikiPathways),"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,3.033183e-09,0.776877,0.717530,0.046217,15.525217,0.85,13.196434,20.752696,14.993823,3.872186
1429169_at,Rbm3,RNA binding motif protein 3 [Source:MGI Symbol...,ENSMUSG00000031167,19652,NaN,translation // response to cold // gene silenc...,RNA binding // nucleic acid binding // nucleot...,NaN,GenMAPP-mRNA_processing_binding_Reactome // mR...,"mmu-miR-1(miRanda), mmu-miR-1192(miRanda), mmu...",...,3.407933e-11,1.526910,1.266540,0.083616,15.147171,0.85,12.875095,19.786399,14.295673,3.780962
1455614_at,Tomm40l,translocase of outer mitochondrial membrane 40...,ENSMUSG00000005674,641376,NaN,ion transport // anion transport // transport ...,voltage-gated anion channel activity,mitochondrion // protein complex // mitochondr...,GenMAPP-Nuclear_Receptors,"mmu-miR-132(miRanda), mmu-miR-146a(miRanda), m...",...,2.515957e-04,0.594447,0.431527,0.029358,14.698850,0.85,12.494023,18.671350,13.490051,3.672880
1434595_at,Trim9,tripartite motif-containing 9 [Source:MGI Symb...,ENSMUSG00000021071,94090,NaN,synaptic vesicle exocytosis,protein binding // metal ion binding // zinc i...,cytoplasm // intracellular // synaptosome,NaN,"mmu-miR-10a(RNAhybrid|miRanda), mmu-miR-10b(RN...",...,5.277075e-17,1.176330,0.673933,0.046850,14.384822,0.85,12.227098,17.910258,12.940161,3.597244


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239868_AK_S_ZT6.CEL',
       'GSM239869_AK_S_ZT6.CEL', 'GSM239870_AK_S_ZT6.CEL', 'avg-AK_S_ZT6',
       'log_fold-AK_S_ZT6_vs_AK_SD_ZT6', 'fold-AK_S_ZT6_vs_AK_SD_ZT6',
       'rawp-AK_S_ZT6_vs_AK_SD_ZT6', 'adjp-AK_S_ZT6_vs_AK_SD_ZT6',
       'GSM239871_AK_S_ZT12.CEL', 'GSM239872_AK_S_ZT12.CEL',
       'GSM239873_AK_S_ZT12.CEL', 'avg-AK_S_ZT12',
       'log_fold-AK_S_ZT12_vs_AK_SD_ZT12', 'fold-AK_S_ZT12_vs_AK_SD_ZT12',
       'rawp-AK_S_ZT12_vs_AK_SD_ZT12', 'adjp-AK_S_ZT12_vs_AK_SD_ZT12',
       'GSM239880_AK_SD_ZT6.CEL', 'GSM239881_AK_SD_ZT6.CEL',
       'GSM239882_AK_SD_ZT6.CEL', 'avg-AK_SD_ZT6', 'GSM239883_AK_SD_ZT12.CEL',
       'GSM239884_AK_SD_ZT12.CEL', 'GSM239885_AK_SD_ZT12.CEL',
       'avg-AK_SD_ZT12', 'GSM239891_B6_S_ZT6.CEL',
  

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM239868_AK_S_ZT6.CEL,GSM239869_AK_S_ZT6.CEL,GSM239870_AK_S_ZT6.CEL,avg-AK_S_ZT6,log_fold-AK_S_ZT6_vs_AK_SD_ZT6,fold-AK_S_ZT6_vs_AK_SD_ZT6,rawp-AK_S_ZT6_vs_AK_SD_ZT6,adjp-AK_S_ZT6_vs_AK_SD_ZT6,GSM239871_AK_S_ZT12.CEL,GSM239872_AK_S_ZT12.CEL,...,ANOVA-adjp,largest fold,430AV2_ZT12_AK_Enrich,430AV2_ZT12_AK_poolStDev,430AV2_ZT12_AK_Cohens_d,430AV2_ZT12_AK_J,430AV2_ZT12_AK_Hedges_g,430AV2_ZT12_AK_Var_d,430AV2_ZT12_AK_Var_g,430AV2_ZT12_AK_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_ZT12_AK_Enrich',u'430AV2_ZT12_AK_Hedges_g', u'430AV2_ZT12_AK_Var_g', u'430AV2_ZT12_AK_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_ZT12_AK_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_ZT12_AK_Enrich,430AV2_ZT12_AK_Hedges_g,430AV2_ZT12_AK_Var_g,430AV2_ZT12_AK_SEg
MGI symbol,,,,
0610005C13Rik,0.038927,0.762882,0.530166,0.728125
0610008F07Rik,-0.119813,-1.421571,0.650072,0.806270
0610009B22Rik,0.033943,0.740585,0.527372,0.726204
0610009D07Rik,-0.107137,-1.433063,0.668499,0.815142
0610009O20Rik,-0.195503,-2.145705,0.865338,0.930235
0610010K14Rik,-0.097153,-0.935640,0.554618,0.744727
0610012G03Rik,0.067529,0.514017,0.635571,0.788004
0610031J06Rik,0.105540,1.890118,0.779379,0.882824
0610037L13Rik,0.022193,0.244059,0.486630,0.697589
